In [13]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import Annotated, Literal, List, Dict, Any
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults

In [14]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI()
llm.invoke("Hey there, how are you?")

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 14, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoLXmIgYn6Sb6KyKWGUgbRTynNIHY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1ac9bf13-ff5c-4df5-8ca6-78f999bdfa8b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 30, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
search_tool = TavilySearchResults(tavily_api_key=TAVILY_API_KEY)

In [16]:
members=["researcher","coder"]
options = members+["FINISH"]

In [17]:
class Router(TypedDict):
    next: Literal["researcher", "coder", "FINISH"]

In [18]:
class State(MessagesState):
    next:str

In [19]:
system_prompt = f""""
You are a supervisor, tasked with managing a conversation between the following workers: {members}. 
Given the following user request, respond with the worker to act next. 
Each worker will perform a task and respond with their results and status. 
When finished, respond with FINISH.
"""

In [ ]:
def supervisor(state:State)->Command[Literal['researcher', 'coder', '__end__']]:
    
    messages = [{"role": "system", "content": system_prompt},] + state["messages"]
    
    llm_with_structure_output = llm.with_structured_output(Router)
    response = llm_with_structure_output.invoke(messages)
    goto=response["next"]
    
    print("**********BELOW IS MY GOTO***************")    
    print(goto)
    
    if goto == "FINISH":
        goto=END
    
    return Command(goto=goto, update={"next":goto})

In [26]:
def researcher(state: State) -> Command[Literal['supervisor', 'coder', 'FINISH']]:
    
    research = create_react_agent(llm, tools=[search_tool], prompt = "You are just a researcher, tasked with finding information based on the user request. Do not do any math")
    result = research.invoke(state)

    return Command(update = {"messages":[HumanMessage(content=result["messages"][-1].content, name="researcher")]}, goto="supervisor")

    pass

In [22]:
def coder(state: State) -> Command[Literal['supervisor', 'coder', 'FINISH']]:
    pass

In [24]:
graph = StateGraph(MessagesState)

graph.add_node('supervisor', supervisor)
graph.add_node('researcher', researcher)
graph.add_node('coder', coder)

graph.add_edge(START, 'supervisor')

app = graph.compile()

app

ValueError: Found edge ending at unknown node `FINISH`